In [1]:
import numpy as np
import h5py
import os
from tqdm.notebook import tqdm

from methods.base import BaseConfig
# from methods.pore_press_calc_functions import find_pore_pressure

In [6]:
# to pore press calc func 

import pde
from pde import DiffusionPDE, ScalarField, UnitGrid, MemoryStorage, movie, PDE, CartesianGrid, VectorField, CylindricalSymGrid

def get_xi(perm, params):
    '''
    perm - np.aaray with permeability
    other is taken from params object

    Функция для подсчёта коэффициента диффузии. Подробности - Баренблатт стр.19
    Размерности:
    1 Мпа = 10**7 г/см*с2
    1 мД = 10**-11 см2
    1 сПз = 10**-2 г/см*с
    Итого размерность [xi] = [мД]*[МПа]/[сПз] =
                           =  10**-11 [см2] * 10**7 [г/см*с2] / 10**-2 [г/см*с] =
                           =  10**-2 [см2/с]
                           =  10**-6 [м2/с]
 
    Function for calculating the diffusion coefficient. Details - Barenblatt p. 19
    Dimensionality:
    1 MPa = 10^7 g/cm*s^2
    1 mD = 10^-11 cm^2
    1 cP = 10^-2 g/cm*s
    Therefore, the dimensionality of [xi] is:
    [xi] = [mD] * [MPa] / [cP] =
         = 10^-11 [cm^2] * 10^7 [g/cms^2] / 10^-2 [g/cms] =
         = 10^-2 [cm^2/s] =
         = 10^-6 [m^2/s]
    '''
    xi = 10**-6 * perm / (params.m0 * params.mu * (1/params.K_ro + 1/params.K_m)) # m^2/s
    return xi * params.time_scale # rescaling to time scale


def get_source_by_Q(params):
    '''
    dP/dt ~ K_m * ((dv/dt)/V) - pressure change when dv of fluid is pumped in volume V
    '''
    V = 2000 * 3.1415 * 0.1**2 / 4  # some "volume" of source (borehole). m^3
    dP = params.K_ro * params.Q / V # pressure change MPa/s
    return dP * params.time_scale # rescaling to time scale

def get_p0(perm, params):
    '''
    initial pressure (without hydristat)
    '''
    return np.ones_like(perm) * params.P0


def find_pore_pressure(perm, params):
    '''
    pore pressure calc
    '''
    dx, dy, dz = params.dx_dy_dz
    x_length, y_length, z_length = params.side_lenght 
    grid = CartesianGrid([[0, x_length], [0, y_length], [0, z_length]], perm.shape)  # generate grid

    p0 = get_p0(perm, params) # initial pore pressure
    pore_pressure_field = pde.ScalarField(grid, data=p0)
 
    source = f"{get_source_by_Q(params)}"  # constant source
    source_field = ScalarField(grid, data=0)
    source_field.insert(params.source_loc, 1)  # one borehole - point source
    # source = f"sin(2*pi*{10}*t)" # давление на скважине как то меняется или...
    
    xi = get_xi(perm, params) # xi
    xi_field = ScalarField(grid, data=xi)

    bc_x = {'derivative': 0}
    bc_y = {'derivative': 0}
    bc_z = {'derivative': 0}
    bc = [bc_x, bc_y, bc_z]

    eq = PDE({'u': f"xi*laplace(u) + {source}*source_field"}, bc=bc,
             consts={'source_field': source_field, 'xi': xi_field})  # уравнение диффузии с источником

    storage = MemoryStorage()

    '''
    Внимание! t_range и dt измеряются относительно заданнго масштаба времени! ТО есть если масштаб был час (3600 сек),
    то dt = 1 это тоже 1 час, а t_range = 24*365 это один год.
    '''
    result = eq.solve(pore_pressure_field, t_range=params.t_range, dt=params.dt, tracker=["plot", storage.tracker(1)])

    return storage.data

In [7]:
perms_path = 'downscaled_models_02_26_2024__16_06_58.h5'

with h5py.File(perms_path, 'r') as f:
    perm = f['perm'][2]

params = BaseConfig(Q = 1, time_scale = 86400, t_range = 30, dt = 0.0001)

pore_press = find_pore_pressure(perm, params)


Output()

In [7]:
params

{'side_lenght': (4000, 4000, 4000), 'shape': (20, 20, 20), 'time_scale': 86400, 't_range': 30, 'dt': 0.0001, 'P0': 1e-07, 'Q': 1, 'm0': 0.2, 'mu': 2, 'K_ro': 10000, 'K_m': 10000, 'dx_dy_dz': (200.0, 200.0, 200.0)}